In [1]:
%run gcal.ipynb


In [2]:
activities = {}

In [3]:
CALENDARS = ["Work", "Meetings"]
SEPARATOR = " - "
CACHE_FILE="/tmp/token.pickle"
CREDENTIALS_FILE="/Users/anna/bin/google_calendar/credentials.json"
PERIODS = ["daily", "weekly", "overall"]


In [4]:
import ipywidgets as widgets
from datetime import date, timedelta, datetime
import plotly.express as px
import plotly.graph_objects as go
from enum import Enum
from plotly.subplots import make_subplots


class TimePeriodType(Enum):
    DAILY = "daily"
    WEEKLY = "weekly"
    OVERALL = "overall"

    @classmethod
    def all_to_list(cls):
        return [cls.DAILY.value, cls.WEEKLY.value, cls.OVERALL.value]

class TimePeriod:
    def __init__(self, start, end, type):
        self.start = start
        self.end = end
        self.type = type


class TimePeriods:
    def __init__(self, start, end, type):
        if type == TimePeriodType.DAILY:
            date_list = [start + timedelta(days=x) for x in range((end - start).days + 1)]
            self.periods = [TimePeriod(d, d, type) for d in date_list]
        elif type == TimePeriodType.WEEKLY:
            self.periods = []
            current_date = start
            while current_date < end:
                day_diff = 6 - current_date.weekday()
                self.periods.append(TimePeriod(current_date, current_date + timedelta(days=day_diff), type))
                current_date = current_date + timedelta(days=day_diff + 1)
            self.periods.append(TimePeriod((self.periods[-1].end + timedelta(days=1)), end, type))
        elif type == TimePeriodType.OVERALL:
            self.periods = [TimePeriod(start, end, type)]
        else:
            raise Exception(f"Unknown time period type {type}")

    def get_period(self, date):
        """Return the time period a date belongs to."""
        for period in self.periods:
            if date >= period.start and date <= period.end:
                return period

        return None

class AggregationType(Enum):
    TOTAL = "total"
    PERCENTAGE = "percentage"

    @classmethod
    def all_to_list(cls):
        return [cls.TOTAL.value, cls.PERCENTAGE.value]

header = widgets.HTML(
    value="""
        <h1>Calendar Insights</h1>
    """
)

start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=date.today() - timedelta(days=28)
)

end_date = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=date.today()
)

calendars = widgets.SelectMultiple(
    options=CALENDARS,
    value=CALENDARS,
    description='Calendars',
    disabled=False
)

period = widgets.Dropdown(
    options=TimePeriodType.all_to_list(),
    value="daily",
    description="Period"
)

aggregation = widgets.Dropdown(
    options=AggregationType.all_to_list(),
    value="total",
    description="Aggregation"
)

separator = widgets.HTML(
    value="<br>"
)

In [5]:
def refresh_activities():
    global activities
    activities = load_calendar_data(calendars.value, SEPARATOR, CREDENTIALS_FILE, CACHE_FILE, datetime.combine(start_date.value, datetime.min.time()), datetime.combine(end_date.value, datetime.min.time()))

refresh_activities()


In [6]:
def calendar_over_time(start_date, end_date, calendars, period, aggregation):
    data_by_calendar = {}
    periods = TimePeriods(start_date, end_date, TimePeriodType(period))
    
    for calendar, calendar_data in activities.items():
        data_by_period = {d.start: 0 for d in periods.periods}

        for activity in calendar_data:
            day_diff = activity.end.date() - activity.start.date()
            current_period = periods.get_period(activity.start.date())
            if current_period is None:
                continue

            current_period = current_period.start
            current_date = activity.start

            for day in range(day_diff.days + 1):
                if datetime.combine(current_date + timedelta(days=1), datetime.min.time()) < activity.end:
                    data_by_period[current_period] += (current_date + timedelta(days=1) - current_date).hours

                    next_period = periods.get_period((current_date + timedelta(days=1)).date())
                    if next_period is None:
                        continue

                    current_date = current_date + timedelta(days=1)
                    if next_period.start != current_period:
                        current_period = next_period.start
                else:   
                    data_by_period[current_period] += (activity.end - current_date).seconds / 3600.0

        data_by_calendar[calendar] = list(data_by_period.values())

    dates = [d.start for d in periods.periods]

    if aggregation == AggregationType.PERCENTAGE.value:
        date_totals = {}
        for c, data in data_by_calendar.items():
            for i, val in enumerate(data):
                if dates[i] not in date_totals:
                    date_totals[dates[i]] = val
                else:
                    date_totals[dates[i]] += val

        data_by_calendar = {c: [v / date_totals[dates[i]] * 100.0 if date_totals[dates[i]] else 0 for i, v in enumerate(data)] for c, data in data_by_calendar.items()}
    
    data_by_calendar["Date"] = dates
  
    if len(list(activities.keys())) > 0:
        fig = px.bar(data_by_calendar, x="Date", y=list(calendars), labels = {"value": "Time spent [h]" if aggregation == AggregationType.TOTAL else "Time spent %"}, title = "Time spent in each category daily", color_discrete_sequence=px.colors.qualitative.Dark24)
        fig.layout.template = "plotly_dark"
        fig.show()



In [7]:
def area_over_time(start_date, end_date, calendars, period, aggregation):
    periods = TimePeriods(start_date, end_date, TimePeriodType(period))
    data_by_category = {}
    
    for calendar, calendar_data in activities.items():
        for activity in calendar_data:
            day_diff = activity.end.date() - activity.start.date()
            current_date = activity.start

            current_period = periods.get_period(activity.start.date())
            if current_period is None:
                continue

            current_period = current_period.start

            if activity.name is None:
                continue
            
            area = activity.name

            if area not in data_by_category:
                data_by_category[area] = {d.start: 0 for d in periods.periods}

            for day in range(day_diff.days + 1):
                if datetime.combine(current_date + timedelta(days=1), datetime.min.time()) < activity.end:
                    data_by_category[area][current_period] += (current_date + timedelta(days=1) - current_date).hours
                    next_period = periods.get_period((current_date + timedelta(days=1)).date())
                    if next_period is None:
                        continue

                    current_date = current_date + timedelta(days=1)
                    if next_period.start != current_period:
                        current_period = next_period.start
                else:   
                    data_by_category[area][current_period] += (activity.end - current_date).seconds / 3600.0

    data_by_category = {category: list(d.values()) for category, d in data_by_category.items()}
    categories = list(data_by_category.keys())
    
    dates = [d.start for d in periods.periods]

    if aggregation == AggregationType.PERCENTAGE.value:
        date_totals = {}
        for c, data in data_by_category.items():
            for i, val in enumerate(data):
                if dates[i] not in date_totals:
                    date_totals[dates[i]] = val
                else:
                    date_totals[dates[i]] += val

        data_by_category = {c: [v / date_totals[dates[i]] * 100.0 if date_totals[dates[i]] else 0 for i, v in enumerate(data)] for c, data in data_by_category.items()}
    
    data_by_category["Date"] = dates

    if len(list(activities.keys())) > 0:
        fig = px.bar(data_by_category, x="Date", y=categories, labels = {"value": "Time spent [h]" if aggregation == AggregationType.TOTAL else "Time spent %"}, title = "Time spent in each area daily", color_discrete_sequence=px.colors.qualitative.Dark24)
        fig.layout.template = "plotly_dark"
        fig.update_layout(
            title=go.layout.Title(
                text="Time spent in each area <br><sup>todo</sup>",
                xref="paper",
                x=0
            )
        )
        fig.show()

In [8]:
def context_switches_over_time(start_date, end_date, calendars, period, aggregation):
    periods = TimePeriods(start_date, end_date, TimePeriodType(period))
    context_switches = {d.start: 0 for d in periods.periods}
    
    for calendar, calendar_data in activities.items():
        for activity in calendar_data:
            day_diff = activity.end.date() - activity.start.date()
            current_date = activity.start

            current_period = periods.get_period(activity.start.date())
            if current_period is None:
                continue

            current_period = current_period.start

            if activity.name is None:
                continue

            for day in range(day_diff.days + 1):
                if datetime.combine(current_date + timedelta(days=1), datetime.min.time()) < activity.end:
                    context_switches[current_period] += 1
                    next_period = periods.get_period((current_date + timedelta(days=1)).date())
                    if next_period is None:
                        continue

                    current_date = current_date + timedelta(days=1)
                    if next_period.start != current_period:
                        current_period = next_period.start
                else:   
                    context_switches[current_period] += 1
    context_switches["Context Switches"] = list(context_switches.values())
    context_switches["Date"] = [d.start for d in periods.periods]

    if len(list(activities.keys())) > 0:
        fig = px.bar(context_switches, x="Date", y="Context Switches", title = "Number of context switches daily", color_discrete_sequence=px.colors.qualitative.Dark24)
        fig.layout.template = "plotly_dark"
        fig.show()

In [9]:
def averages(start_date, end_date, calendars, period, aggregation):
    date_diff = (end_date - start_date).days + 1
    activities_prev_period = load_calendar_data(
        CALENDARS, 
        SEPARATOR, 
        CREDENTIALS_FILE, 
        CACHE_FILE, 
        datetime.combine(start_date - timedelta(days=date_diff), datetime.min.time()), datetime.combine(end_date - timedelta(days=date_diff), datetime.min.time())
    )

    def total_time_tracked(activities):
        total_time = 0
        for calendar, calendar_data in activities.items():
            for activity in calendar_data:
                total_time += activity.duration

        return total_time

    def total_context_switches(activities):
        context_switches = 0
        for calendar, calendar_data in activities.items():
            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_date = activity.start

                if activity.name is None:
                    continue

                for day in range(day_diff.days + 1):
                    if datetime.combine(current_date.date() + timedelta(days=1), datetime.min.time()) < activity.end:
                        context_switches += 1
                        current_date = current_date.date() + timedelta(days=1)
                    else:   
                        context_switches += 1
        return context_switches

    context_switches = total_context_switches(activities)
    prev_context_switches = total_context_switches(activities_prev_period)

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "domain"}, {"type": "domain"}]],
)
    fig.layout.template = "plotly_dark"

    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = context_switches / date_diff,
        title = "Average Context Switches",
        delta = {'reference': prev_context_switches / date_diff, 'relative': True, 'position' : "bottom"}),
        row=1, col=1,
    )

    time_tracked = total_time_tracked(activities)
    prev_time_tracked = total_time_tracked(activities_prev_period)

    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = time_tracked / 3600,
        title = "Total Time Tracked [h]",
        delta = {'reference': prev_time_tracked / 3600, 'relative': True, 'position' : "bottom"}),
        row=1, col=2,
    )
    fig.update_layout(
        height = 300,
        width = 700
    )
    fig.show()

In [10]:
def refresh(start_date, end_date, calendars, period, aggregation):
    refresh_activities()
    
    averages(start_date=start_date, end_date=end_date, calendars=calendars, period=period, aggregation=aggregation)

    calendar_over_time(start_date=start_date, end_date=end_date, calendars=calendars, period=period, aggregation=aggregation)
    area_over_time(start_date=start_date, end_date=end_date, calendars=calendars, period=period, aggregation=aggregation)
    context_switches_over_time(start_date=start_date, end_date=end_date, calendars=calendars, period=period, aggregation=aggregation)

widget = widgets.interactive(refresh, start_date=start_date, end_date=end_date, calendars=calendars, period=period, aggregation=aggregation)
controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(widgets.VBox([header, controls, separator, output]))



interactive(children=(DatePicker(value=datetime.date(2022, 10, 26), description='Start Date'), DatePicker(valu…